In [1]:
from soundtrack import *
import time
import sqlite3
from IPython.display import clear_output
import IPython
from datetime import datetime, timedelta
import pickle
import datetime as dt
filename='music_dict.p'
buffer_sleep=0 #use if laggy connection to prevent STs from get skipped.
#pickle will have new shuffle songs, but use only when necessary because only created four times per day 
#and will have songs already heard today in many cases.
try:
    music_dict=music_dict
    print("*******No pickle needed*********")
except:
    print("*******Pickle needed*********")
    pickle=pickle.load(open(filename,"rb"))
    music_dict=pickle
while datetime.now().hour < 24:
    for song in music_dict:
        #print(song)
        b=song[1][2]
        current_datetime=datetime.now().strftime('%A %H:%M:%S')
        #b=float(getMediaDuration(song[0]))
        con = sqlite3.connect('soundtrack.db')
        print("Current Time:{}".format(current_datetime))
        table='song_schedule'
        current_dow=datetime.now().strftime('%w')
        current_time=datetime.now().strftime('%H:%M:%S')
        statement="SELECT * FROM {} WHERE dow='{}' AND time >'{}' ORDER BY dow ASC, time ASC LIMIT 0,1".format(table,current_dow,current_time)
        cursor=con.execute(statement)
        rows = cursor.fetchall()
        for row in rows:
            #print(row)
            next_song_time=datetime.strptime(row[1],'%H:%M:%S').time()
        song_dt = dt.datetime.combine(dt.date.today(), next_song_time)
        now=dt.datetime.now()
        difference=song_dt-now
        difference=difference.seconds
        print("{} seconds until next Soundtrack song.".format(difference))
        end_tm=datetime.now() + timedelta(0,float(3000))
        end_tm=end_tm.strftime('%H:%M:%S')
        beg_tm=datetime.now() + timedelta(0,float(-3000))
        beg_tm=beg_tm.strftime('%H:%M:%S')
        statement="SELECT * FROM {} WHERE dow={} ORDER BY dow ASC, time ASC".format(table,current_dow)
        cursor=con.execute(statement)
        rows = cursor.fetchall()
        i=0
        for row2 in rows:
            if str(beg_tm) < row2[1] < str(end_tm):
                #print a few st songs before and after now
                if ((str(current_time) < row2[1]) and i==0):
                    print("Next: {}".format(row2[0:3]))
                    i=i+1
                else:
                    print(row2[0:3])
        #time algos
        if difference < 60: #play ST song
            print("Less than 60 seconds")
            time.sleep(difference/2) #play sooner. maybe /2 half if sometimes wait seems tooo long
            song="/home/pi/Music/"+row[2]
            b=float(getMediaDuration(song))
            print("{} is {} seconds.".format(song,b))
            IPython.display.display(IPython.display.Audio(song,autoplay=True, embed=None))            
            time.sleep(b+buffer_sleep+2) #an additional 2 seconds for STs to play until the end
            #clear_output()
            break
        elif difference < 300: #find a shuffle song that fits this short gap before next ST
            if 1 < difference-b <= 15:
                music_dict.remove(song) #remove shuffle song about to play so that next time start program it skips
                IPython.display.display(IPython.display.Audio(song[0],autoplay=True, embed=None))
                print("Now Playing(pre-ST): {} : {} seconds : ".format(getMediaName(song[0]),b), end="")
                next_tm = datetime.now() + timedelta(0,float(b))
                print("Ends @ {}".format(next_tm.time()))
                statement=getShuffleCountExists(song[0])
                print(statement)
                con.execute(statement)
                con.commit()
                con.close()
                time.sleep(b+buffer_sleep)
                break
        elif b < difference: #play shuffle song  
            music_dict.remove(song)
            IPython.display.display(IPython.display.Audio(song[0],autoplay=True, embed=None))
            print("Now Playing: {} : {} seconds : ".format(getMediaName(song[0]),b), end="")
            next_tm = datetime.now() + timedelta(0,float(b))
            print("Ends @ {}".format(next_tm.time()))
            statement=getShuffleCountExists(song[0])
            print(statement)
            con.execute(statement)
            con.commit()
            con.close()
            time.sleep(b+buffer_sleep)
        clear_output()

*******Pickle needed*********


FileNotFoundError: [Errno 2] No such file or directory: 'music_dict.p'

In [ ]:
print(music_dict)